<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/thegym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[tf_gan_tpus](https://github.com/bxck75/gan/blob/master/tensorflow_gan/examples/colab_notebooks/tfgan_on_tpus.ipynb)

[tf_gan](https://github.com/bxck75/gan/blob/master/tensorflow_gan/examples/colab_notebooks/tfgan_tutorial.ipynb)

In [0]:
import os,subprocess, re
# system functions
def recursive_glob(treeroot, pattern):
  results = []
  for base, dirs, files in os.walk(treeroot):
    goodfiles = fnmatch.filter(files, pattern)
    results.extend(os.path.join(base, f) for f in goodfiles)
  return results

def cmdline(cmd,pr=False):
  result =[]
  f = os.popen(cmd)
  try:
    for line in f:
      if pr == True:
        print(line)
        result.append(line),
  finally:
    f.close()
  return result

# stinking default folder...poef!
cmdline('rm -r /content/sample_data')
# cmdline('pip install gitpython')
# pick fs
filesystem = "local"
if filesystem == 'gdrive':
  # sync google drive
  from google.colab import drive
  import os
  # if drive is needed uncomment
  print('Google drive ',end='')
  drive.mount('/content/drive',force_remount=True)
  print('Root folder set to ',end='')
  root = '/content/drive/My drive/image_learning'
  print(root)
else:
  print('Root folder set to ',end='')
  root = '/content/image_learning'
  print(root) 

#shoot root
os.makedirs(root,exist_ok = True)
os.chdir(root)

# install missing libs
cmdline('pip install sklearn imageio tensorflow-gpu==2.0.0-alpha0')
cmdline('apt install tree')
# cmdline('pip install -q --upgrade --force-reinstall tensorflow-gpu')

# import the rest
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.optimizers import SGD, RMSprop,adam
from keras.utils import np_utils
import theano, glob, shutil, fnmatch, time, cv2, itertools
from IPython.display import clear_output, display, Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from subprocess import PIPE, Popen
import tensorflow as tf

# list of reps uncomment to install
reps=[
    'Zardinality/Tiled-Matrix-Multiplication-in-Cuda', 
    'drallensmith/neat-python',
    'CodeReclaimers/neat-python',
    'uber-research/deep-neuroevolution',
    'MorvanZhou/Evolutionary-Algorithm',
    'llSourcell/blockchain-python-tutorial',
    'llSourcell/autoencoder_demo',
    'llSourcell/autoencoder_explained',
    'llSourcell/awesome-public-datasets',
    'llSourcell/7_Research_Directions_Deep_Learning',
    'hardmaru/sketch-rnn',
    'hardmaru/sketch-rnn-datasets',
    'hardmaru/sketch-rnn-poste',
    'Gogul09/virtual-drum',
    'ytdl-org/youtube-dl',
    'nnUyi/GAN-Collections',
    'carpedm20/BEGAN-tensorflow',
    'mikf/gallery-dl',
    'NVlabs/stylegan',
    'mauriceling/dose',
    'corenel/Realistic-Neural-Talking-Head-Models',
    'hardmaru/estool',
    'deepmipt/DeepPavlov',
    'tjwei/Flappy-Turtle.',
    'tjwei/fonttools',
    'tjwei/blender3d_import_psk_psa',
    'lllyasviel/sketchKeras',
    'Mckinsey666/Anime-Face-Dataset',
    'chenyuntc/pytorch-book',
    'lllyasviel/style2paints',
    'llSourcell/GANS-for-style-transfer',
    'opencv/open_model_zoo',
    'hindupuravinash/the-gan-zoo',
    'corenel/GAN-Zoo',
    'eriklindernoren/Keras-GAN',
    'junyanz/CycleGAN',
    'junyanz/pytorch-CycleGAN-and-pix2pix',
    'junyanz/iGAN', #----> !wget http://efrosgans.eecs.berkeley.edu/iGAN/datasets/church_64.zip <----dataset 	outdoor_128.zip 	handbag_128.zip !!!
    'martinarjovsky/WassersteinGAN',
    'shaoanlu/faceswap-GAN',
    'LantaoYu/SeqGAN',
    'tjwei/GANotebooks',
    'adeshpande3/Tensorflow-Programs-and-Tutorials',
    'adeshpande3/Generative-Adversarial-Networks',
    'adeshpande3/KaggleGhosts',
    'adeshpande3/OpenAI_Gym_Universe',
    'diegoalejogm/gans',
    'osh/KerasGAN',
    'r9y9/gantts',
    'jayleicn/animeGAN',
    'jayleicn/ImageNet-Training',
    'Zardinality/WGAN-tensorflow',
    'tree-gan/BonsaiNet',
    'RossMcKenzie/TreeGAN',
    'carpedm20/BEGAN-tensorflow',
    'martinarjovsky/WassersteinGAN',
    'timsainb/Tensorflow-MultiGPU-VAE-GAN',
    'Larox/python-moviepy-meetup',
    'tjwei/keras-yolo3',
    'tensorflow/gan',
    'tensorflow/moonlight'
    'tensorflow/models',
    'tensorflow/datasets',
    'tensorflow/docs',
    'mnicnc404/CartoonGan-tensorflow',
    'Yijunmaverick/CartoonGAN-Test-Pytorch-Torch',
    'keras-team/keras-contrib',
    'mnicnc404/CartoonGan-tensorflow',
    'AlekseyLobanov/neural-life',
]


import os,sys,pandas as pd
#@title Helper voor functies
# helper voor gspread functionaliteit
need_help_on_module=pd
# toon de methods voor de module
help=pd.DataFrame.from_records([[a for a in dir(need_help_on_module) if '__' not in a]])
display(help)



list_homebrew=[]
def findFilesInFolderYield(path,  extension, containsTxt='', subFolders = True, excludeText = ''):
    if type(containsTxt) == str: # if a string and not in a list
        containsTxt = [containsTxt]

    myregexobj = re.compile('\.' + extension + '$')    # Makes sure the file extension is at the end and is preceded by a .
    try:   # Trapping a OSError or FileNotFoundError:  File permissions problem I believe
        for entry in os.scandir(path):
            if entry.is_file() and myregexobj.search(entry.path): # 
                bools = [True for txt in containsTxt if txt in entry.path and (excludeText == '' or excludeText not in entry.path)]
                if len(bools)== len(containsTxt):
                    yield entry.stat().st_size, entry.stat().st_atime_ns, entry.stat().st_mtime_ns, entry.stat().st_ctime_ns, entry.path
            elif entry.is_dir() and subFolders:   # if its a directory, then repeat process as a nested function
                yield from findFilesInFolderYield(entry.path,  extension, containsTxt, subFolders)
    except OSError as ose:
        print('Cannot access ' + path +'. Probably a permissions error ', ose)
    except FileNotFoundError as fnf:
        print(path +' not found ', fnf)        
# Gitgo class
class GitGo(): 
  def __init__(self,repos,sub_repos=False,chdir=True,path='/content/'):
    self.sub_repo_list = []
    self.GitUsers=[]
    self.repos = repos
    self.chdir = chdir
    self.path = path
    os.makedirs(self.path, exist_ok = True)
    if 'help' in self.repos:
      self.help()
    self.pull_reps()
    self.custom_reps_setup()
    if sub_repos == True:
      self.get_other_reps()
  def help(self):
    return "* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo(<list of reps to install>, sub_repos=<True/False, chdir=<True/False>, path=<root path>)"
        #--pull al selected reps  
    def pull_reps(self):    
        for rep in self.repos:
            self.rep=rep.split('/')
                # change folder check
                if self.chdir ==True:
                    #Switch to path
                    os.chdir(self.path)
                    # pull the git repo
                    cmdline('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git',True)
                    # Set the return value for rep rootpath
                    self.path=self.path+self.rep[1]
            # show imported files
    os.system('ls ' + root)  
    #--repository required dependancies setup
   
  def custom_reps_setup(self):
    # cyclegan
    if 'pytorch-CycleGAN-and-pix2pix' in self.repos:
      !export PYTHONPATH={root}/pytorch-CycleGAN-and-pix2pix:$PYTHONPATH
      !pip install dominate
    # custom stuff for CartoonGAN-tensorflow and keras-team/keras-contrib   
    if 'keras-team/keras-contrib' in self.repos:
      os.chdir(self.path+'/keras-contrib')
      cmdline('python convert_to_tf_keras.py')
      cmdline('USE_TF_KERAS=1')
      cmdline('python setup.py install')
      import tensorflow as tf
      tf.__version__     
    # custom setup stuff for gallery-dl repo   
    if 'mikf/gallery-dl' in self.repos:
      os.chdir(root+'/gallery-dl')
      cmdline("pip install -e . |grep 'succes'",True)
    # custom setup stuff for youtube-dl repo    
    if 'ytdl-org/youtube-dl' in self.repos:
      os.chdir(root+'/youtube-dl') 
      cmdline("pip install -e . |grep 'succes'",True)      
    # switch backt to root
    os.chdir(self.path) 
    
#--grab the username if a repos is installed
#--generate a file of all other reps of users
  def get_other_reps(self):          
    for r in self.repos:
      self.GUSER=r.split('/')[0]
      self.repo_name=r.split('/')[1]
      self.GitUsers.append(self.GUSER)      
      !curl https://api.github.com/users/{self.GUSER}/repos?per_page=100 \
            | grep -o 'git@[^"]*' > {root}/info.txt
      cmdline('cat '+root+"/info.txt |awk -F ':' '{print $2}'|awk -F '.' '{print $1}' \
          > "+self.path+"/"+self.GUSER+"_repositories.txt",True)      
      with open(root+'/info.txt','r') as f:
        for line in f:
          cline=line.split(':')[1].split('.')[0]
          self.sub_repo_list.append(cline),  
            #--return path and sub reps
  def __repr__(self):
    return self.path,self.sub_repo_list
# Hauling Ass!
GR=GitGo(reps,sub_repos=True,path=root)


clear_output()

In [0]:
%cd piss-ant-pix2pix

##Get images and make the dataset

In [0]:
# # get pailkids images
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/My\ Drive/Colab\ Notebooks/garbagepailkids.zip /content/installed_resp/piss-ant-pix2pix/
# !unzip /content/installed_resp/piss-ant-pix2pix/garbagepailkids.zip
# clear_output()

In [0]:
!rm -r /content/gallery-dl
!git clone https://github.com/mikf/gallery-dl.git
%cd /content/gallery-dl

In [0]:
# !gallery-dl -d music_dump/ 20 
!python gallery-dl --help


In [0]:
!sudo rm -r /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp


In [0]:
!python tools/process.py --input_dir /content/installed_resp/piss-ant-pix2pix/garbagepailkids --operation resize --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized
clear_output()

In [0]:
!sudo mkdir -p /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank


In [0]:
!python tools/process.py --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --operation blank --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank
clear_output()

In [0]:
!ls /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank


In [0]:
!mkdir -p /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized/_combined


In [0]:
!python tools/process.py --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --b_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank --operation combine --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized/_combined
clear_output()

In [0]:
!python tools/split.py --dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized/_combined
clear_output()

In [0]:
!sudo mkdir -p /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize


In [0]:
!cp -rf /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized/_combined/ /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/
clear_output()

In [0]:
!rm -r /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp

In [0]:
# TRAIN!!!!!
output_folder='/content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training'
input_folder = '/content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/_combined/train'
training_epochs=10
!python pix2pix.py --mode train --output_dir {output_folder}_train --max_epochs {training_epochs} --input_dir {input_folder} --which_direction BtoA

In [0]:
# checkpoints
!ls -l /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training_train

In [0]:
!wget https://live.staticflickr.com/2936/33549790582_48a514baed_b.jpg

In [0]:
# /content/installed_resp/piss-ant-pix2pix/33549790582_48a514baed_b.jpg
!python pix2pix.py --mode test --output_dir /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/test --input_dir /content/installed_resp/piss-ant-pix2pix --seed 0 --checkpoint /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training_train


In [0]:
# save the complete model
!python /content/installed_resp/piss-ant-pix2pix/server/tools/export-checkpoint.py --checkpoint /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/_combined/train --output_file /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training_train/garbagepailkids_BtoA_model_save.bin

In [0]:
# store zip to drive
!zip /content/installed_resp/piss-ant-pix2pix/garbagepailkids/* /content/installed_resp/piss-ant-pix2pix/garbagepailkids_original_images.zip
!zip /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/_combined/* /content/installed_resp/piss-ant-pix2pix/garbagepailkids_dataset.zip
!zip /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training_train/garbagepailkids_BtoA_model_save.bin /content/installed_resp/piss-ant-pix2pix/garbagepailkids_model_bin.zip
!zip /content/installed_resp/piss-ant-pix2pix/garbagepailkids_resize/training_train/* /content/installed_resp/piss-ant-pix2pix/garbagepailkids.zip

%cd /content/drive/My\ Drive/Colab\ Notebooks/Best_Colabs
!cp /content/installed_resp/piss-ant-pix2pix/garbagepailkids_original_images.zip garbagepailkids_original_images.zip
!cp /content/installed_resp/piss-ant-pix2pix/garbagepailkids_dataset.zip ./garbagepailkids_dataset.zip
!cp /content/installed_resp/piss-ant-pix2pix/garbagepailkids_model_bin.zip ./garbagepailkids_model_bin.zip
!cp /content/installed_resp/piss-ant-pix2pix/garbagepailkids.zip ./garbagepailkids.zip

## pretrained
